# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846828074800                   -0.70    5.0
  2   -7.852321600517       -2.26       -1.53    1.0
  3   -7.852645998240       -3.49       -2.52    3.2
  4   -7.852646676607       -6.17       -3.35    2.5
  5   -7.852646685894       -8.03       -4.75    1.2
  6   -7.852646686727       -9.08       -5.23    4.2
  7   -7.852646686730      -11.52       -5.89    2.0
  8   -7.852646686730      -12.51       -7.49    2.0
  9   -7.852646686730      -14.35       -7.94    2.8
 10   -7.852646686730   +  -14.75       -8.78    2.0
 11   -7.852646686730   +  -15.05       -9.54    1.0
 12   -7.852646686730   +  -14.75       -8.68    1.0
 13   -7.852646686730      -14.45       -8.23    2.0
 14   -7.852646686730   +  -14.75       -8.50    1.0
 15   -7.852646686730      -14.75       -8.47    1.0
 16   -7.852646686730   +  -14.75       -8.72    1.0
 17   -7.852646686730   +    -Inf       -9.36 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846822901025                   -0.70           4.8
  2   -7.852525669876       -2.24       -1.63   0.80    2.0
  3   -7.852636758820       -3.95       -2.70   0.80    1.0
  4   -7.852646412189       -5.02       -3.29   0.80    2.2
  5   -7.852646682706       -6.57       -4.13   0.80    1.8
  6   -7.852646686496       -8.42       -4.75   0.80    1.2
  7   -7.852646686723       -9.64       -5.68   0.80    1.8
  8   -7.852646686730      -11.15       -6.62   0.80    1.8
  9   -7.852646686730      -12.72       -7.75   0.80    2.2
 10   -7.852646686730      -14.57       -8.54   0.80    2.2
 11   -7.852646686730   +  -15.05       -8.82   0.80    1.8
 12   -7.852646686730   +  -14.57      -10.06   0.80    1.2
 13   -7.852646686730      -14.45      -10.53   0.80    2.8
 14   -7.852646686730   +  -14.57      -11.00   0.80    1.0
 15   -7.852646686730      -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.425407e+01     3.697795e+00
 * time: 0.40921497344970703
     1     1.553112e+00     1.757882e+00
 * time: 0.6309118270874023
     2    -1.663573e+00     2.016311e+00
 * time: 0.6550998687744141
     3    -3.992693e+00     1.698892e+00
 * time: 0.6895308494567871
     4    -5.472235e+00     1.418458e+00
 * time: 0.7240908145904541
     5    -7.024048e+00     1.074440e+00
 * time: 0.7581667900085449
     6    -7.198445e+00     1.489547e+00
 * time: 0.782526969909668
     7    -7.550130e+00     1.209519e+00
 * time: 0.8068759441375732
     8    -7.603539e+00     2.271987e+00
 * time: 0.8307688236236572
     9    -7.641082e+00     2.114389e+00
 * time: 0.944598913192749
    10    -7.664945e+00     1.959445e+00
 * time: 0.9792218208312988
    11    -7.681865e+00     1.832239e+00
 * time: 1.013206958770752
    12    -7.687211e+00     1.733765e+00
 * time: 1.048292875289917
    13    -7.742562e+00     5.169872e-01
 * time: 1.0926878452301

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846824935523                   -0.70    4.8
  2   -7.852323021312       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686712                   -2.55
  2   -7.852646686730      -10.75       -6.01
  3   -7.852646686730   +  -14.75      -12.67


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.531913212229115e-14
|ρ_newton - ρ_scfv| = 1.1408466299350108e-12
|ρ_newton - ρ_dm|   = 6.114159494888362e-10
